# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [7]:
#1)1.
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

# Criar a variável resposta: log(renda)
df['log_renda'] = np.log(df['renda'])

# Verificar as categorias mais frequentes para usar como referência
print("Categorias mais frequentes:")
print("Sexo:", df['sexo'].value_counts().idxmax())
print("Tipo de renda:", df['tipo_renda'].value_counts().idxmax())
print("Educação:", df['educacao'].value_counts().idxmax())
print("Estado civil:", df['estado_civil'].value_counts().idxmax())
print("Tipo de residência:", df['tipo_residencia'].value_counts().idxmax())

Categorias mais frequentes:
Sexo: F
Tipo de renda: Assalariado
Educação: Secundário
Estado civil: Casado
Tipo de residência: Casa


In [8]:
#1)2.
# Definir a fórmula usando Patsy (mantendo categorias mais frequentes como referência)
formula_inicial = ('log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + '
                   'C(tipo_renda, Treatment(reference="Assalariado")) + '
                   'C(educacao, Treatment(reference="Secundário")) + '
                   'C(estado_civil, Treatment(reference="Casado")) + '
                   'C(tipo_residencia, Treatment(reference="Casa")) + '
                   'idade + tempo_emprego + qt_pessoas_residencia')

# Ajustar o modelo
modelo_inicial = smf.ols(formula_inicial, data=df).fit()

# Exibir resumo do modelo
print(modelo_inicial.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 15 Jun 2025   Prob (F-statistic):               0.00
Time:                        20:42:22   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

#1)3.
Os resultados sugerem que:

  - Homens têm renda significativamente maior que mulheres

  - Tempo de emprego é o fator mais importante para renda

  - Educação superior completa aumenta a renda em relação ao ensino médio

  - Estado civil diferente de casado está associado a maior renda

  - Posse de imóvel e veículo são indicadores positivos de renda

In [14]:
#2)1.
def refine_model(initial_model, data, max_iter=20):
    current_model = initial_model
    removed_vars = []
    
    for i in range(max_iter):
        # Obter p-values e remover o Intercept
        pvalues = current_model.pvalues.drop('Intercept')
        
        # Verificar se todas as variáveis são significativas
        if all(pvalues <= 0.05):
            print("\nTodas as variáveis restantes são significativas (p < 0.05)")
            break
            
        # Identificar a variável menos significativa
        least_sig_var = pvalues.idxmax()
        least_sig_pvalue = pvalues.max()
        
        print(f"\nIteração {i+1}:")
        print(f"Variável menos significativa: {least_sig_var} (p-value = {least_sig_pvalue:.4f})")
        
        # Criar nova fórmula sem a variável menos significativa
        current_formula = current_model.model.formula
        new_formula = current_formula.replace(least_sig_var, '')
        
        # Limpar a fórmula (remover '++' ou '+ ' extras)
        new_formula = new_formula.replace('+ +', '+').strip()
        while '  ' in new_formula:
            new_formula = new_formula.replace('  ', ' ')
        
        # Ajustar novo modelo
        new_model = smf.ols(new_formula, data=data).fit()
        
        # Verificar se o modelo piorou significativamente
        if new_model.rsquared_adj < current_model.rsquared_adj - 0.01:
            print(f"Atenção: Remoção de {least_sig_var} reduziu o R² ajustado em mais de 1%")
            print("Interrompendo o refinamento para evitar perda de qualidade do modelo")
            break
            
        # Atualizar o modelo atual
        removed_vars.append(least_sig_var)
        current_model = new_model
        
        # Exibir resumo a cada 5 iterações
        if (i+1) % 5 == 0:
            print(f"\nModelo após {i+1} iterações:")
            print(current_model.summary())
    
    print("\nVariáveis removidas durante o processo:")
    for var in removed_vars:
        print(f"- {var}")
    
    return current_model

# Aplicar o refinamento ao modelo inicial
modelo_final = refine_model(modelo_inicial, df)


Iteração 1:
Variável menos significativa: C(educacao, Treatment(reference="Secundário"))[T.Primário] (p-value = 0.8444)

Iteração 2:
Variável menos significativa: C(educacao, Treatment(reference="Secundário"))[T.Primário] (p-value = 0.8444)

Iteração 3:
Variável menos significativa: C(educacao, Treatment(reference="Secundário"))[T.Primário] (p-value = 0.8444)

Iteração 4:
Variável menos significativa: C(educacao, Treatment(reference="Secundário"))[T.Primário] (p-value = 0.8444)

Iteração 5:
Variável menos significativa: C(educacao, Treatment(reference="Secundário"))[T.Primário] (p-value = 0.8444)

Modelo após 5 iterações:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 15 Jun 2025   Prob (F-stati

In [16]:
#2)2.
# Comparação de métricas
comparacao = pd.DataFrame({
    'Modelo Inicial': [
        modelo_inicial.rsquared_adj,
        modelo_inicial.aic,
        modelo_inicial.bic,
        len(modelo_inicial.params)
    ],
    'Modelo Final': [
        modelo_final.rsquared_adj,  # Corrigido: usando modelo_final em vez de modelo_atual
        modelo_final.aic,
        modelo_final.bic,
        len(modelo_final.params)
    ]
}, index=['R² Ajustado', 'AIC', 'BIC', 'Nº de Parâmetros'])

print("\nComparação entre modelos:")
print(comparacao)


Comparação entre modelos:
                  Modelo Inicial  Modelo Final
R² Ajustado             0.356227      0.356227
AIC                 27185.298825  27185.298825
BIC                 27370.989495  27370.989495
Nº de Parâmetros       25.000000     25.000000


# 3
Avaliação dos modelos:

Modelo Inicial:

Contém todas as variáveis disponíveis

Algumas variáveis podem não ser significativas

Pode estar superajustado (overfitting)

Modelo Final:

Contém apenas variáveis significativas (p < 0.05)

Possivelmente mais parcimonioso

Coeficientes mais estáveis